In [4]:
from pandas.io.sql import read_sql
from pyiem.plot import MapPlot
from pyiem.util import get_dbconn
import matplotlib.pyplot as plt

In [2]:
pgconn = get_dbconn('postgis')
df = read_sql("""with data as (select ugc,
 valid - lag(valid) OVER (PARTITION by ugc ORDER by valid ASC) as delta, hour01 from ffg)
 select ugc, substr(ugc, 3, 1) as ztype,
 extract(epoch from sum(case when hour01 < 1. then delta else '0 minutes'::interval end))
 / extract(epoch from sum(delta) + '1 minute'::interval) * 100. as p from data
 GROUP by ugc, ztype""", pgconn, index_col='ugc')

In [14]:
mp = MapPlot(sector='conus', continentalcolor='white',
            title='2001-2016 Percentage of Time 1 Hour Flash Flood Guidance < 1 inch',
            subtitle=("Estimated amount of Rainfall "
                      "needed for non-urban Flash Flooding to commence"))
cmap = plt.get_cmap('gist_rainbow_r')
cmap.set_under('tan')
cmap.set_over('pink')
df2 = df[df['ztype'] == 'C']
bins = range(0, 11, 1)
bins[0] = 0.01
mp.fill_ugcs(df2['p'].to_dict(), bins, cmap=cmap, plotmissing=False,
               ilabel=False)
df2 = df[df['ztype'] == 'Z']
mp.fill_ugcs(df2['p'].to_dict(), bins, cmap=cmap, plotmissing=False,
               units='Percentage', ilabel=False)
mp.postprocess(filename='test.png')